# xG model implementation

##### This code is borrowed heavily from https://github.com/Friends-of-Tracking-Data-FoTD/SoccermaticsForPython for the data treatment section

In [ ]:
#The basics
import pandas as pd
import numpy as np
import json

#Plotting
import matplotlib.pyplot as plt
import FCPython 

#Statistical fitting of models
import statsmodels.api as sm
import statsmodels.formula.api as smf

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import ClusterCentroids
import seaborn as sns
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix

##### Single run code

In [ ]:
'''
#Decide which league to load
#Wyscout data from https://figshare.com/collections/Soccer_match_event_dataset/4415000/2
with open('All_data.json') as f: #All_data is a dataset consisting of the fusion of all 7 competitions
    data = json.load(f)

train = pd.DataFrame(data)

shots=train[train['subEventName']=='Shot']

shots_model=pd.DataFrame(columns=['Goal','X','Y'])

#Go through the dataframe and calculate X, Y co-ordinates.
#Distance from a line in the centre
#Shot angle.
#Details of tags can be found here: https://apidocs.wyscout.com/matches-wyid-events
for i,shot in shots.iterrows():
    
    header=0
    for shottags in shot['tags']:
        if shottags['id']==403:
            header=1
    #Only include non-headers        
    if not(header):        
        shots_model.at[i,'X']=100-shot['positions'][0]['x']
        shots_model.at[i,'Y']=shot['positions'][0]['y']
        shots_model.at[i,'C']=abs(shot['positions'][0]['y']-50)
    
        #Distance in metres and shot angle in radians.
        x=shots_model.at[i,'X']*105/100
        y=shots_model.at[i,'C']*65/100
        shots_model.at[i,'Distance']=np.sqrt(x**2 + y**2)
        a = np.arctan(7.32 *x /(x**2 + y**2 - (7.32/2)**2))
        if a<0:
            a=np.pi+a
        shots_model.at[i,'Angle'] =a
    
        #Was it a goal
        shots_model.at[i,'Goal']=0
        for shottags in shot['tags']:
                #Tags contain that its a goal
                if shottags['id']==101:
                    shots_model.at[i,'Goal']=1


shots_model.to_csv('shots.csv', index = False)
'''

In [ ]:
data = pd.read_csv('shots.csv') #Load dataset with all shots from the 7 competitions

In [ ]:
data

In [ ]:
data.describe()

In [ ]:
x = data.drop(['Goal','X','Y','C'], axis=1)
y = data['Goal']

In [ ]:
sns.lmplot(x= 'Distance', 
           y= 'Angle', 
           height=15,
           aspect=1,
           data=data, 
           fit_reg=False, 
           hue= 'Goal', 
           legend=True)

plt.xlabel("Distance")
plt.ylabel("Angle")
 
plt.show()

##### Data standardization (not used as it yielded worst results)

In [ ]:
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#x = scaler.fit_transform(x)

In [ ]:
# Train/test split
from sklearn.model_selection import train_test_split 
  
xtrain, xtest, ytrain, ytest = train_test_split( 
    x, y, test_size=0.25, random_state=123)

#### We implement oversampling and undersampling techniques (SMOTE, ADASYN and Cluster Centroids) until we have full class balance

In [ ]:
oversample = SMOTE(random_state = 123)
xtrain_smote, ytrain_smote = oversample.fit_resample(xtrain, ytrain)

In [ ]:
oversample = ADASYN(sampling_strategy = 0.5, random_state = 123)
xtrain_ada, ytrain_ada = oversample.fit_resample(xtrain, ytrain)

In [ ]:
cc = ClusterCentroids(random_state = 123)
xtrain_res, ytrain_res = cc.fit_resample(xtrain, ytrain)

### Logistic Regression models

In [ ]:
from sklearn.linear_model import LogisticRegression 

classifier = LogisticRegression(random_state = 123) 
classifier.fit(xtrain, ytrain)
y_pred = classifier.predict(xtest)
y_score = classifier.predict_proba(xtest)[:, 1]
print ("Accuracy : ", accuracy_score(ytest, y_pred))
print("F1Score : ",f1_score(ytest, y_pred, average="weighted"))
print("AUC Score : ", roc_auc_score(ytest, y_score))
precision, recall, _= precision_recall_curve(ytest, y_score) 
pr_auc = auc(recall, precision)
print("PR-AUC Score:", pr_auc)
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, y_score)
# get the best threshold
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold=%f' % (best_thresh))
threshold = best_thresh
y_pred = (y_score >= best_thresh).astype(int)
print(confusion_matrix(ytest, y_pred))
print('Accuracy with custom threshold:' , accuracy_score(ytest, y_pred))
print('F1Score with custom threshold:' , f1_score(ytest, y_pred, average="weighted"))
print('Coefficients:' ,classifier.coef_)
print('Intercept:' ,classifier.intercept_)

In [ ]:
classifier = LogisticRegression(random_state = 123) 
classifier.fit(xtrain_smote, ytrain_smote)
y_pred = classifier.predict(xtest)
y_score = classifier.predict_proba(xtest)[:, 1]
print ("Accuracy : ", accuracy_score(ytest, y_pred))
print("F1Score : ",f1_score(ytest, y_pred, average="weighted"))
print("AUC Score : ", roc_auc_score(ytest, y_score))
precision, recall, _= precision_recall_curve(ytest, y_score) 
pr_auc = auc(recall, precision)
print("PR-AUC Score:", pr_auc)
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, y_score)
# get the best threshold
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold=%f' % (best_thresh))
threshold = best_thresh
y_pred = (y_score >= best_thresh).astype(int)
print(confusion_matrix(ytest, y_pred))
print('Accuracy with custom threshold:' , accuracy_score(ytest, y_pred))
print('F1Score with custom threshold:' , f1_score(ytest, y_pred, average="weighted"))
print('Coefficients:' ,classifier.coef_)
print('Intercept:' ,classifier.intercept_)

In [ ]:
classifier = LogisticRegression(random_state = 123) 
classifier.fit(xtrain_ada, ytrain_ada)
y_pred = classifier.predict(xtest)
y_score = classifier.predict_proba(xtest)[:, 1]
print ("Accuracy : ", accuracy_score(ytest, y_pred))
print("F1Score : ",f1_score(ytest, y_pred, average="weighted"))
print("AUC Score : ", roc_auc_score(ytest, y_score))
precision, recall, _= precision_recall_curve(ytest, y_score) 
pr_auc = auc(recall, precision)
print("PR-AUC Score:", pr_auc)
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, y_score)
# get the best threshold
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold=%f' % (best_thresh))
threshold = best_thresh
y_pred = (y_score >= best_thresh).astype(int)
print(confusion_matrix(ytest, y_pred))
print('Accuracy with custom threshold:' , accuracy_score(ytest, y_pred))
print('F1Score with custom threshold:' , f1_score(ytest, y_pred, average="weighted"))
print('Coefficients:' ,classifier.coef_)
print('Intercept:' ,classifier.intercept_)

In [ ]:
classifier = LogisticRegression(random_state = 123) 
classifier.fit(xtrain_res, ytrain_res)
y_pred = classifier.predict(xtest)
y_score = classifier.predict_proba(xtest)[:, 1]
print ("Accuracy : ", accuracy_score(ytest, y_pred))
print("F1Score : ",f1_score(ytest, y_pred, average="weighted"))
print("AUC Score : ", roc_auc_score(ytest, y_score))
precision, recall, _= precision_recall_curve(ytest, y_score) 
pr_auc = auc(recall, precision)
print("PR-AUC Score:", pr_auc)
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, y_score)
# get the best threshold
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold=%f' % (best_thresh))
threshold = best_thresh
y_pred = (y_score >= best_thresh).astype(int)
print(confusion_matrix(ytest, y_pred))
print('Accuracy with custom threshold:' , accuracy_score(ytest, y_pred))
print('F1Score with custom threshold:' , f1_score(ytest, y_pred, average="weighted"))
print('Coefficients:' ,classifier.coef_)
print('Intercept:' ,classifier.intercept_)

### XGBOOST models

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

In [ ]:
# Create an instance of the XGBClassifier
model = XGBClassifier(objective='binary:logistic', base_score = 0.1) #Base score of 0.1 accounts for class imbalance

# Fit the model to the training data
model.fit(xtrain, ytrain)
y_pred = model.predict(xtest)
y_score = model.predict_proba(xtest)[:, 1]
print ("Accuracy : ", accuracy_score(ytest, y_pred))
print("F1Score : ",f1_score(ytest, y_pred, average="weighted"))
print("AUC Score : ", roc_auc_score(ytest, y_score))
precision, recall, _= precision_recall_curve(ytest, y_score) 
pr_auc = auc(recall, precision)
print("PR-AUC Score:", pr_auc)
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, y_score)
# get the best threshold
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold=%f' % (best_thresh))
threshold = best_thresh
y_pred = (y_score >= best_thresh).astype(int)
print(confusion_matrix(ytest, y_pred))
print('Accuracy with custom threshold:' , accuracy_score(ytest, y_pred))
print('F1Score with custom threshold:' , f1_score(ytest, y_pred, average="weighted"))

In [ ]:
# Create an instance of the XGBClassifier
model = XGBClassifier(objective='binary:logistic')

# Fit the model to the training data
model.fit(xtrain_smote, ytrain_smote)
y_pred = model.predict(xtest)
y_score = model.predict_proba(xtest)[:, 1]
print ("Accuracy : ", accuracy_score(ytest, y_pred))
print("F1Score : ",f1_score(ytest, y_pred, average="weighted"))
print("AUC Score : ", roc_auc_score(ytest, y_score))
precision, recall, _= precision_recall_curve(ytest, y_score) 
pr_auc = auc(recall, precision)
print("PR-AUC Score:", pr_auc)
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, y_score)
# get the best threshold
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold=%f' % (best_thresh))
threshold = best_thresh
y_pred = (y_score >= best_thresh).astype(int)
print(confusion_matrix(ytest, y_pred))
print('Accuracy with custom threshold:' , accuracy_score(ytest, y_pred))
print('F1Score with custom threshold:' , f1_score(ytest, y_pred, average="weighted"))

In [ ]:
# Create an instance of the XGBClassifier
model = XGBClassifier(objective='binary:logistic')

# Fit the model to the training data
model.fit(xtrain_ada, ytrain_ada)
y_pred = model.predict(xtest)
y_score = model.predict_proba(xtest)[:, 1]
print ("Accuracy : ", accuracy_score(ytest, y_pred))
print("F1Score : ",f1_score(ytest, y_pred, average="weighted"))
print("AUC Score : ", roc_auc_score(ytest, y_score))
precision, recall, _= precision_recall_curve(ytest, y_score) 
pr_auc = auc(recall, precision)
print("PR-AUC Score:", pr_auc)
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, y_score)
# get the best threshold
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold=%f' % (best_thresh))
threshold = best_thresh
y_pred = (y_score >= best_thresh).astype(int)
print(confusion_matrix(ytest, y_pred))
print('Accuracy with custom threshold:' , accuracy_score(ytest, y_pred))
print('F1Score with custom threshold:' , f1_score(ytest, y_pred, average="weighted"))

In [ ]:
# Create an instance of the XGBClassifier
model = XGBClassifier(objective='binary:logistic')

# Fit the model to the training data
model.fit(xtrain_res, ytrain_res)
y_pred = model.predict(xtest)
y_score = model.predict_proba(xtest)[:, 1]
print ("Accuracy : ", accuracy_score(ytest, y_pred))
print("F1Score : ",f1_score(ytest, y_pred, average="weighted"))
print("AUC Score : ", roc_auc_score(ytest, y_score))
precision, recall, _= precision_recall_curve(ytest, y_score) 
pr_auc = auc(recall, precision)
print("PR-AUC Score:", pr_auc)
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, y_score)
# get the best threshold
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold=%f' % (best_thresh))
threshold = best_thresh
y_pred = (y_score >= best_thresh).astype(int)
print(confusion_matrix(ytest, y_pred))
print('Accuracy with custom threshold:' , accuracy_score(ytest, y_pred))
print('F1Score with custom threshold:' , f1_score(ytest, y_pred, average="weighted"))